### Lab3 - Mnożenie macierzy hierarchicznych przez wektor i przez siebie (Jan Masternak, Jakub Mróz):

#### Kwestie techniczne:

* Pomiary wykonywane na procesorze i5-13400f będą miały przed bezpośrednio przed rozszerzeniem końcówkę "...i5_13400f"

##### Importy potrzebnych paczek:

Poniższe wystarczy jednorazowo odkomentować i uruchomić.

In [18]:
using Pkg
Pkg.add("CSV")
Pkg.add("DataFrames")
Pkg.add("Plots")
Pkg.add("LinearAlgebra")
Pkg.add("LaTeXStrings")
Pkg.add("Images")
Pkg.add("ImageIO")
Pkg.add("ColorTypes")
Pkg.add("ImageView")
Pkg.add("Printf")
Pkg.add("WebP")
Pkg.add("Statistics");

: 

Poniższe uruchamiamy każdorazowo.

In [4]:
using CSV
using DataFrames
using Plots
using LinearAlgebra
using LaTeXStrings
using Images
using ImageIO
using ColorTypes
using ImageView
using Printf
using Statistics;

In [ ]:
cpu = "apple_M1_Pro"
error_eps = 1e-2

In [6]:
import Base: +, *, -, size, iterate

In [ ]:
mutable struct CompressedMatrixNode
    rank::Union{Int64, Nothing}
    addr::Union{Tuple{Int64, Int64, Int64, Int64}, Nothing} 

    left_upper_child::Union{CompressedMatrixNode, Nothing}
    left_lower_child::Union{CompressedMatrixNode, Nothing}
    right_upper_child::Union{CompressedMatrixNode, Nothing}
    right_lower_child::Union{CompressedMatrixNode, Nothing}

    singular_values
    U_matrix::Union{Matrix{Float64}, Nothing}
    V_tr_matrix::Union{Matrix{Float64}, Nothing}

end


function size(cmn::CompressedMatrixNode)
    return (cmn.addr[2]-cmn.addr[1]+1, cmn.addr[4]-cmn.addr[3]+1)
end


function size(cmn::CompressedMatrixNode, dim::Int64)
    if dim == 1
        return cmn.addr[2]-cmn.addr[1]+1
    elseif dim == 2
        return cmn.addr[4]-cmn.addr[3]+1
    else 
        throw(ArgumentError("Compressed matrix only has two dimansions")) 
    end 
end


function is_compressed(cmn::CompressedMatrixNode)
    return isnothing(cmn.left_upper_child) && isnothing(cmn.left_lower_child) && 
           isnothing(cmn.right_upper_child) && isnothing(cmn.right_lower_child) && 
           !isnothing(cmn.U_matrix)
end


function change_addr_to_mock(cmn::Union{CompressedMatrixNode, Nothing})
    if isnothing(cmn1) 
        return
    end

    new_r_max, new_c_max = size(cmn)
    cmn.addr = (1, new_r_max, 1, new_c_max)
end


function create_new_compressed_matrix_node()::CompressedMatrixNode
    return CompressedMatrixNode(
        nothing, nothing,
        nothing, nothing, nothing, nothing,
        nothing, nothing, nothing
    )
end


function break_up_compressed_cmn(cmn::CompressedMatrixNode)
    r_min, r_max, c_min, c_max = cmn.addr
    r_mid = div(r_min+r_max,2)
    c_mid = div(c_min+c_max,2)
    right_upper_child, left_lower_child, right_lower_child = nothing, nothing, nothing

    gamma = cmn.rank
    U_border = r_mid - r_min + 1
    V_border = c_mid - c_min + 1
    U_end = r_max - r_min + 1
    V_end = c_max - c_min + 1

    left_upper_child = create_new_compressed_matrix_node()
    left_upper_child.addr = (r_min, r_mid, c_min, c_mid)
    left_upper_child.rank = gamma
    left_upper_child.U_matrix = cmn.U_matrix[1:U_border , 1:end]
    if !isnothing(cmn.V_tr_matrix)
        if r_min == r_mid && c_min == c_mid 
            left_upper_child.U_matrix *= cmn.V_tr_matrix[1:end , 1:V_border]
        else
            left_upper_child.V_tr_matrix = cmn.V_tr_matrix[1:end , 1:V_border]
        end
    end

    if U_border < U_end 
        left_lower_child = create_new_compressed_matrix_node()
        left_lower_child.addr = (r_mid+1, r_max, c_min, c_mid)
        left_lower_child.rank = gamma 
        left_lower_child.U_matrix = cmn.U_matrix[U_border+1:end , 1:end]
        if !isnothing(cmn.V_tr_matrix)
            if r_mid+1 == r_max && c_min == c_mid
                left_lower_child.U_matrix *= cmn.V_tr_matrix[1:end , 1:V_border]
            else
                left_lower_child.V_tr_matrix = cmn.V_tr_matrix[1:end , 1:V_border]
            end
        end

        if V_border < V_end
            right_lower_child = create_new_compressed_matrix_node()
            right_lower_child.addr = (r_mid+1, r_max, c_mid+1, c_max)
            right_lower_child.rank = gamma 
            right_lower_child.U_matrix = cmn.U_matrix[U_border+1:end , 1:end]
            if !isnothing(cmn.V_tr_matrix)
                if r_mid+1 == r_max && c_mid+1 == c_max
                    right_lower_child.U_matrix *= cmn.V_tr_matrix[1:end , V_border+1:end]
                else
                    right_lower_child.V_tr_matrix = cmn.V_tr_matrix[1:end , V_border+1:end]
                end
            end

        end
    end
    
    if V_border < V_end
        right_upper_child = create_new_compressed_matrix_node()
        right_upper_child.addr = (r_min, r_mid, c_mid+1, c_max)
        right_upper_child.rank = gamma 
        right_upper_child.U_matrix = cmn.U_matrix[1:U_border , 1:end]
        if !isnothing(cmn.V_tr_matrix)
            if r_min == r_mid && c_mid+1 == c_max
                right_upper_child.U_matrix *= cmn.V_tr_matrix[1:end , V_border+1:end]
            else
                right_upper_child.V_tr_matrix = cmn.V_tr_matrix[1:end , V_border+1:end]
            end
        end

    end

    return left_upper_child, left_lower_child, right_upper_child, right_lower_child
end


function fix_addrs(cmn::Union{CompressedMatrixNode, Nothing}, r_min::Int64, r_max::Int64, c_min::Int64, c_max::Int64)
    if isnothing(cmn)
        return 
    end

    cmn.addr = (r_min, r_max, c_min, c_max)

    r_mid = div(r_min+r_max, 2)
    c_mid = div(c_min+c_max, 2)
    fix_addrs(cmn.left_upper_child, r_min, r_mid, c_min, c_mid)
    fix_addrs(cmn.left_lower_child, r_mid+1, r_max, c_min, c_mid)
    fix_addrs(cmn.right_upper_child, r_min, r_mid, c_mid+1, c_max)
    fix_addrs(cmn.right_lower_child, r_mid+1, r_max, c_mid+1, c_max)

end


mutable struct CompressedMatrix
    head::CompressedMatrixNode
    size::Tuple{Int64, Int64}

end


function size(cmn::CompressedMatrix)
    return cmn.size
end


function size(cmn::CompressedMatrix, dim::Int64)
    return cmn.size[dim]
end


function fix_addrs(cm::CompressedMatrix)
    fix_addrs(cm.head, 1, size(cm, 1), 1, size(cm, 2))
end

In [ ]:
function trunc_svd(A::Matrix{Float64}; gamma = min(size(A,1), size(A,2)))
    gamma = min(gamma, size(A,1), size(A,2))
    U, D, V = svd(A)
    D = diagm(D)
    U = U[: , 1:gamma]
    D = D[1:gamma , 1:gamma]
    V = V[: , 1:gamma]
    return U, D, V'
end


function compress_submatrix(A, r_min, r_max, c_min, c_max, U, D, V_tr, gamma)
    if all(abs.(A[r_min:r_max , c_min:c_max]) .< error_eps)
        v = create_new_compressed_matrix_node()
        v.rank = 0
        v.addr = (r_min, r_max, c_min, c_max)
        return v
    else
        sigmas = diag(D)
        v = create_new_compressed_matrix_node()
        v.rank = gamma
        v.addr = (r_min, r_max, c_min, c_max)
        v.singular_values = sigmas[1:gamma]
        v.U_matrix = U[: , 1:gamma]
        v.V_tr_matrix = D[1:gamma , 1:gamma] * V_tr[1:gamma , :]
        return v
    end
end


function compress_matrix(A, r_min=1, r_max=size(A,1), c_min=1, c_max=size(A,2); gamma=1, eps=1)
    if r_min == r_max && c_min == c_max
        v = create_new_compressed_matrix_node()
        v.rank = 1
        v.addr = (r_min, r_max, c_min, c_max)
        v.U_matrix = reshape([A[r_min, c_min]], 1, 1)
        return v
    end   
    
    U, D, V_tr = trunc_svd(A[r_min:r_max , c_min:c_max], gamma=gamma+1)
    if gamma+1 > size(D,1)
        gamma = size(D,1)
        if D[gamma, gamma] - eps < error_eps
            v = compress_submatrix(A, r_min, r_max, c_min, c_max, U, D, V_tr, gamma)
            return v 
        end
    elseif D[gamma+1, gamma+1] - eps < error_eps
        v = compress_submatrix(A, r_min, r_max, c_min, c_max, U, D, V_tr, gamma)
        return v
    end 

    r_mid = div(r_min+r_max,2)
    c_mid = div(c_min+c_max,2)
    right_upper_child, left_lower_child, right_lower_child = nothing, nothing, nothing
    left_upper_child = compress_matrix(A, r_min, r_mid, c_min, c_mid, gamma=gamma, eps=eps)
    if c_mid < c_max
        right_upper_child = compress_matrix(A, r_min, r_mid, c_mid+1, c_max, gamma=gamma, eps=eps)
    end
    if r_mid < r_max
        left_lower_child = compress_matrix(A, r_mid+1, r_max, c_min, c_mid, gamma=gamma, eps=eps)
        if c_mid < c_max
            right_lower_child = compress_matrix(A, r_mid+1, r_max, c_mid+1, c_max, gamma=gamma, eps=eps)
        end            
    end

    v = create_new_compressed_matrix_node()
    v.addr = (r_min, r_max, c_min, c_max)
    v.left_upper_child = left_upper_child
    v.right_upper_child = right_upper_child
    v.left_lower_child = left_lower_child
    v.right_lower_child = right_lower_child
    return v

end


function perform_matrix_compression(A, r_min=1, r_max=size(A,1), c_min=1, c_max=size(A,2); gamma=1, eps=1)
    return CompressedMatrix(compress_matrix(A, r_min, r_max, c_min, c_max; gamma=gamma, eps=eps), size(A))
end


function build_matrix_based_on_tree(target_matrix, node)
    r_min, r_max, c_min, c_max = node.addr

    if !isnothing(node.U_matrix)
        if !isnothing(node.V_tr_matrix)
            target_matrix[r_min:r_max , c_min:c_max] += node.U_matrix*node.V_tr_matrix
        else
            target_matrix[r_min:r_max , c_min:c_max] += node.U_matrix;
        end
    end

    if !isnothing(node.left_upper_child)
        build_matrix_based_on_tree(target_matrix, node.left_upper_child)
    end

    if !isnothing(node.right_upper_child)
        build_matrix_based_on_tree(target_matrix, node.right_upper_child)
    end
    
    if !isnothing(node.left_lower_child)
        build_matrix_based_on_tree(target_matrix, node.left_lower_child)
    end
    
    if !isnothing(node.right_lower_child)
        build_matrix_based_on_tree(target_matrix, node.right_lower_child)
    end
    
end


function build_matrix_from_compressed(matrix_size, compressed_matrix)
    built_matrix = zeros(matrix_size[1], matrix_size[2])
    build_matrix_based_on_tree(built_matrix, compressed_matrix.head)
    return built_matrix
end


function +(cmn1::Union{CompressedMatrixNode, Nothing}, cmn2::Union{CompressedMatrixNode, Nothing})::Union{CompressedMatrixNode, Nothing}
    if isnothing(cmn1) && isnothing(cmn2)
        return nothing   
    elseif isnothing(cmn1) 
        return deepcopy(cmn2)
    elseif isnothing(cmn2)
        return deepcopy(cmn1)          
    elseif !all(size(cmn1) .== size(cmn2))
        throw(ArgumentError("Cannot add matrix nodes of different sizes"))
    end

    add_cmn::Union{CompressedMatrixNode, Nothing} = nothing

    if cmn1.addr[1] == cmn1.addr[2] && cmn1.addr[3] == cmn1.addr[4]
        add_cmn = create_new_compressed_matrix_node()
        add_cmn.U_matrix = cmn1.U_matrix + cmn2.U_matrix
        add_cmn.rank = cmn1.rank
        add_cmn.addr = cmn1.addr
    
    elseif cmn1.rank == 0 && cmn2.rank == 0
        add_cmn = create_new_compressed_matrix_node()
        add_cmn.rank = 0
        add_cmn.addr = cmn1.addr

    elseif cmn1.rank == 0
        add_cmn = deepcopy(cmn2)
       
    elseif cmn2.rank == 0
        add_cmn = deepcopy(cmn1)    
    
    elseif is_compressed(cmn1) && is_compressed(cmn2) 
        u_wave = [cmn1.U_matrix cmn2.U_matrix]
        v_wave = [cmn1.V_tr_matrix ; cmn2.V_tr_matrix]
        gamma = max(cmn1.rank, cmn2.rank)
        if isnothing(cmn1.U_matrix) || isnothing(cmn2.U_matrix) || isnothing(cmn1.V_tr_matrix) || isnothing(cmn2.V_tr_matrix)
            println(cmn1.U_matrix, "\n\n", cmn2.U_matrix, "\n\n", cmn1.V_tr_matrix, "\n\n", cmn2.V_tr_matrix, "\n\n\n")
        end
        u_dash, d_dash, v_tr_dash = trunc_svd(u_wave*v_wave, gamma=gamma)
        sigmas = diag(d_dash)
        add_cmn = create_new_compressed_matrix_node()
        add_cmn.rank = gamma
        add_cmn.addr = cmn1.addr
        add_cmn.singular_values = sigmas[1:gamma]
        add_cmn.U_matrix = u_dash
        add_cmn.V_tr_matrix = d_dash*v_tr_dash
    
    elseif !is_compressed(cmn1) && !is_compressed(cmn2)
        add_cmn = create_new_compressed_matrix_node()
        add_cmn.rank = cmn1.rank
        add_cmn.addr = cmn1.addr
        add_cmn.left_upper_child = cmn1.left_upper_child + cmn2.left_upper_child
        add_cmn.left_lower_child = cmn1.left_lower_child + cmn2.left_lower_child
        add_cmn.right_upper_child = cmn1.right_upper_child + cmn2.right_upper_child
        add_cmn.right_lower_child = cmn1.right_lower_child + cmn2.right_lower_child

    elseif is_compressed(cmn1) && !is_compressed(cmn2)
        cmn1_left_upper, cmn1_left_lower, cmn1_right_upper, cmn1_right_lower = break_up_compressed_cmn(cmn1)
        add_cmn = create_new_compressed_matrix_node()
        add_cmn.rank = cmn1.rank 
        add_cmn.addr = cmn1.addr 
        add_cmn.left_upper_child = cmn1_left_upper + cmn2.left_upper_child
        add_cmn.left_lower_child = cmn1_left_lower + cmn2.left_lower_child
        add_cmn.right_upper_child = cmn1_right_upper + cmn2.right_upper_child
        add_cmn.right_lower_child = cmn1_right_lower + cmn2.right_lower_child

    elseif !is_compressed(cmn1) && is_compressed(cmn2)
        cmn2_left_upper, cmn2_left_lower, cmn2_right_upper, cmn2_right_lower = break_up_compressed_cmn(cmn2)
        add_cmn = create_new_compressed_matrix_node()
        add_cmn.rank = cmn2.rank 
        add_cmn.addr = cmn2.addr 
        add_cmn.left_upper_child = cmn1.left_upper_child + cmn2_left_upper
        add_cmn.left_lower_child = cmn1.left_lower_child + cmn2_left_lower
        add_cmn.right_upper_child = cmn1.right_upper_child + cmn2_right_upper
        add_cmn.right_lower_child = cmn1.right_lower_child + cmn2_right_lower
    end

    return add_cmn

end


function remove_unneccessary_children(cmn::Union{CompressedMatrixNode, Nothing})::Union{CompressedMatrixNode, Nothing}
    if isnothing(cmn)
        return nothing
    end

    new_cmn = cmn 
    while all(size(new_cmn) .== (1,1)) && isnothing(new_cmn.U_matrix) && !isnothing(new_cmn.left_upper_child)
        new_cmn = cmn.left_upper_child
    end

    return new_cmn
end


function *(cmn1::Union{CompressedMatrixNode, Nothing}, cmn2::Union{CompressedMatrixNode, Nothing})::Union{CompressedMatrixNode, Nothing}
    if isnothing(cmn1) || isnothing(cmn2)
        return nothing
    elseif size(cmn1, 2) != size(cmn2, 1)
        throw(ArgumentError("Cannot multiply compressed matrix nodes of incompatible shapes"))
    end

    mult_cmn::Union{CompressedMatrixNode, Nothing} = nothing
    
    if all(size(cmn1) .== (1,1)) && all(size(cmn2) .== (1,1))
        mult_cmn = create_new_compressed_matrix_node()
        mult_cmn.addr = (1, 1, 1, 1)
        mult_cmn.rank = 1
        mult_cmn.U_matrix = cmn1.U_matrix * cmn2.U_matrix

    elseif cmn1.rank == 0 || cmn2.rank == 0
        mult_cmn = create_new_compressed_matrix_node()
        mult_cmn.rank = 0
        mult_cmn.addr = (1, size(cmn1, 1), 1, size(cmn2, 2))

    elseif is_compressed(cmn1) && is_compressed(cmn2)
        mult_cmn = create_new_compressed_matrix_node()
        V1_tr_matrix = cmn1.V_tr_matrix
        U_dash = nothing
        if !isnothing(V1_tr_matrix)
            U_dash = cmn1.U_matrix * (V1_tr_matrix * cmn2.U_matrix)
        else
            U_dash = cmn1.U_matrix * cmn2.U_matrix
        end
        mult_cmn.addr = (1, size(cmn1, 1), 1, size(cmn2, 2))
        mult_cmn.U_matrix = U_dash
        mult_cmn.V_tr_matrix = deepcopy(cmn2.V_tr_matrix)   
        if sum(Int64.(size(mult_cmn.U_matrix)) .== (1,1)) == 1 && isnothing(mult_cmn.V_tr_matrix)
            mult_cmn.V_tr_matrix = reshape([1], 1, 1)
        end
        mult_cmn.rank = size(mult_cmn.U_matrix, 2)

    elseif !is_compressed(cmn1) && !is_compressed(cmn2)
        mult_cmn = create_new_compressed_matrix_node()
        mult_cmn.left_upper_child = cmn1.left_upper_child * cmn2.left_upper_child + cmn1.right_upper_child * cmn2.left_lower_child
        mult_cmn.right_upper_child = cmn1.left_upper_child * cmn2.right_upper_child + cmn1.right_upper_child * cmn2.right_lower_child
        mult_cmn.left_lower_child = cmn1.left_lower_child * cmn2.left_upper_child + cmn1.right_lower_child * cmn2.left_lower_child
        mult_cmn.right_lower_child = cmn1.left_lower_child * cmn2.right_upper_child + cmn1.right_lower_child * cmn2.right_lower_child
        mult_cmn.addr = (1, size(cmn1, 1), 1, size(cmn2, 2))
        
    elseif is_compressed(cmn1) && !is_compressed(cmn2)
        mult_cmn = create_new_compressed_matrix_node()
        cmn1_left_upper, cmn1_left_lower, cmn1_right_upper, cmn1_right_lower = break_up_compressed_cmn(cmn1)
        mult_cmn.left_upper_child = cmn1_left_upper * cmn2.left_upper_child + cmn1_right_upper * cmn2.left_lower_child
        mult_cmn.right_upper_child = cmn1_left_upper * cmn2.right_upper_child + cmn1_right_upper * cmn2.right_lower_child
        mult_cmn.left_lower_child = cmn1_left_lower * cmn2.left_upper_child + cmn1_right_lower * cmn2.left_lower_child
        mult_cmn.right_lower_child = cmn1_left_lower * cmn2.right_upper_child + cmn1_right_lower * cmn2.right_lower_child
        mult_cmn.addr = (1, size(cmn1, 1), 1, size(cmn2, 2))

    elseif !is_compressed(cmn1) && is_compressed(cmn2)
        mult_cmn = create_new_compressed_matrix_node()
        cmn2_left_upper, cmn2_left_lower, cmn2_right_upper, cmn2_right_lower = break_up_compressed_cmn(cmn2)
        mult_cmn.left_upper_child = cmn1.left_upper_child * cmn2_left_upper + cmn1.right_upper_child * cmn2_left_lower
        mult_cmn.right_upper_child = cmn1.left_upper_child * cmn2_right_upper + cmn1.right_upper_child * cmn2_right_lower
        mult_cmn.left_lower_child = cmn1.left_lower_child * cmn2_left_upper + cmn1.right_lower_child * cmn2_left_lower
        mult_cmn.right_lower_child = cmn1.left_lower_child * cmn2_right_upper + cmn1.right_lower_child * cmn2_right_lower
        mult_cmn.addr = (1, size(cmn1, 1), 1, size(cmn2, 2))

    end

    mult_cmn = remove_unneccessary_children(mult_cmn)

    return mult_cmn

end


function *(cmn::Union{CompressedMatrixNode, Nothing}, alpha::Number)::Union{CompressedMatrixNode, Nothing}
    if isnothing(cmn)
        return nothing 
    end

    if cmn.rank == 0
        return cmn 
    elseif is_compressed(cmn)
        cmn.U_matrix .*= alpha
    else 
        cmn.left_upper_child = cmn.left_upper_child*alpha 
        cmn.left_lower_child = cmn.left_lower_child*alpha
        cmn.right_upper_child = cmn.right_upper_child*alpha
        cmn.right_lower_child = cmn.right_lower_child*alpha
    end

    return cmn
end


function *(alpha::Number, cmn::Union{CompressedMatrixNode, Nothing})::Union{CompressedMatrixNode, Nothing}
    return cmn * alpha
end


function +(cm1::CompressedMatrix, cm2::CompressedMatrix)
    if !all(cm1.size .== cm2.size) 
        throw(ArgumentError("Cannot add matrixes of different sizes"))
    end
    return CompressedMatrix(cm1.head + cm2.head, cm1.size)
end


function *(cm1::CompressedMatrix, cm2::CompressedMatrix)
    if cm1.size[2] != cm2.size[1]
        throw(ArgumentError("Cannot multiply matrixes of incompatible shapes"))
    end
    mult_cm = CompressedMatrix(cm1.head * cm2.head,  (cm1.size[1], cm2.size[2]))
    fix_addrs(mult_cm)
    return mult_cm
end


function *(cm::CompressedMatrix, alpha::Number)
    return CompressedMatrix(alpha * deepcopy(cm.head), size(cm.head))
end


function *(alpha::Number, cm::CompressedMatrix)
    return cm * alpha
end


function -(cm::CompressedMatrix)
    return cm * -1
end


function -(cm1::CompressedMatrix, cm2::CompressedMatrix)
    return cm1 + (-cm2)
end


function *(v::CompressedMatrixNode, X::Vector{Float64})::Vector{Float64}
    if isnothing(v.left_upper_child) && isnothing(v.right_upper_child) && isnothing(v.left_lower_child) && isnothing(v.right_lower_child)
        if v.rank == 0
            return zeros(size(X, 1))
        end

        if !isnothing(v.V_tr_matrix)
            return v.U_matrix * (v.V_tr_matrix * X)
        end

        return v.U_matrix * X
    end

    rows = size(X, 1)
    X1 = X[1:div(rows + 1, 2)]
    X2 = X[div(rows + 1, 2)+1:rows]


    if !isnothing(v.left_upper_child) && size(X1, 1) != 0 
        Y11 = v.left_upper_child * X1
    else
        Y11 = zeros(1)
    end

    if !isnothing(v.right_upper_child) && size(X2, 1) != 0
        Y12 = v.right_upper_child * X2
    else
        Y12 = zeros(1)
    end

    if !isnothing(v.left_lower_child) && size(X1, 1) != 0
        Y21 = v.left_lower_child * X1
    else
        Y21 = zeros(1)
    end

    if !isnothing(v.right_lower_child) && size(X2, 1) != 0
        Y22 = v.right_lower_child * X2
    else
        Y22 = zeros(1)
    end

    if size(v, 1) == 1
        return Y11 + Y12
    end

    if size(v, 2) == 1
        return vcat(Y11, Y21)
    end

    return vcat(Y11 + Y12, Y21 + Y22)
end

function *(v::CompressedMatrix, X::Vector{Float64})
    if v.size[2] != size(X, 1)
        throw(ArgumentError("Incompatible shapes, inner dimensions must agree"))
    end
    return v.head * X
end

In [ ]:
function generate_random_matrix(m, n, a, b)
    A = zeros(m, n)
    for i in 1:1:m 
        for j in 1:1:n 
            A[i,j] = a + (b-a)*rand()
        end
    end
    return A
end

function generate_random_vector(m, a, b)
    A = zeros(m)
    for i in 1:1:m 
        A[i] = a + (b-a)*rand()
    end
    return A
end


function are_equal(A, B)
    if !all(size(A) .== size(B))
        return false
    end

    return all(abs.(A-B) .< error_eps)

end


function test_matrix_compression(low_m, high_m, low_n, high_n, a, b, tests, gamma, eps)
    correct_outputs = 0
    incorrect_ouputs = 0
    for _ in 1:1:tests 
        A = generate_random_matrix(rand(low_m:1:high_m), rand(low_n:1:high_n), a, b)
        cmn = perform_matrix_compression(A, gamma=gamma, eps=eps)
        built_A = build_matrix_from_compressed(size(A), cmn)
        if are_equal(A, built_A)
            correct_outputs += 1
        else 
            incorrect_ouputs += 1    
        end
    end

    return correct_outputs, incorrect_ouputs
end


function run_compression_tests()
    low_m_for_test, high_m_for_test, low_n_for_test, high_n_for_test = 15, 50, 15, 50
    a_for_test, b_for_test = 0, 50
    correct, incorrect = test_matrix_compression(low_m_for_test, high_m_for_test, low_n_for_test, high_n_for_test, a_for_test, b_for_test, 15, 1, 0.1)
    println(correct, " ", incorrect)
end    


function test_compressed_matrix_addition(low_m, high_m, low_n, high_n, a, b, tests, gamma, eps)
    correct_outputs = 0
    incorrect_outputs = 0
    for _ in 1:1:tests 
        m, n = rand(low_m:1:high_m), rand(low_n:1:high_n)
        A = generate_random_matrix(m, n, a, b)
        B = generate_random_matrix(m, n, a, b)
        cm_A = perform_matrix_compression(A, gamma=gamma, eps=eps)
        cm_B = perform_matrix_compression(B, gamma=gamma, eps=eps)
        cm_added = cm_A + cm_B
        AB = A+B
        built_addded = build_matrix_from_compressed(size(cm_added), cm_added)
        if are_equal(A+B, built_addded)
            correct_outputs += 1
        else 
            incorrect_outputs += 1    
        end
    end

    return correct_outputs, incorrect_outputs
end


function run_compressed_addition_tests()
    low_m_for_test, high_m_for_test, low_n_for_test, high_n_for_test = 15, 50, 15, 50
    a_for_test, b_for_test = 0, 50
    correct, incorrect = test_compressed_matrix_addition(low_m_for_test, high_m_for_test, low_n_for_test, high_n_for_test, a_for_test, b_for_test, 15, 1, 0.1)
    println(correct, " ", incorrect)
end


function test_compressed_matrix_multiplication(low_m, high_m, low_n, high_n, a, b, tests, gamma, eps)
    correct_outputs = 0
    incorrect_outputs = 0
    for _ in 1:1:tests 
        m, n, p = rand(low_m:1:high_m), rand(low_n:1:high_n), rand(low_m:1:high_m)
        A = generate_random_matrix(m, n, a, b)
        B = generate_random_matrix(n, p, a, b)
        cm_A = perform_matrix_compression(A, gamma=gamma, eps=eps)
        cm_B = perform_matrix_compression(B, gamma=gamma, eps=eps)
        cm_mult = cm_A * cm_B
        AB = A*B
        built_mult = build_matrix_from_compressed(size(cm_mult), cm_mult)
        if are_equal(A*B, built_mult)
            correct_outputs += 1
        else 
            incorrect_outputs += 1    
        end
    end

    return correct_outputs, incorrect_outputs
end


function run_compressed_multiplication_tests()
    low_m_for_test, high_m_for_test, low_n_for_test, high_n_for_test = 15, 50, 15, 50
    a_for_test, b_for_test = 0, 50
    correct, incorrect = test_compressed_matrix_multiplication(low_m_for_test, high_m_for_test, low_n_for_test, high_n_for_test, a_for_test, b_for_test, 15, 1, 0.1)
    println(correct, " ", incorrect)
end


function test_compressed_matrix_scalar_multiplication(low_m, high_m, low_n, high_n, a, b, tests, gamma, eps)
    correct_outputs = 0
    incorrect_outputs = 0
    for _ in 1:1:tests 
        m, n = rand(low_m:1:high_m), rand(low_n:1:high_n), rand(low_m:1:high_m)
        A = generate_random_matrix(m, n, a, b)
        cm_A = perform_matrix_compression(A, gamma=gamma, eps=eps)
        scalar = a + (b-a)*rand()
        cm_A = cm_A * scalar
        built_mult = build_matrix_from_compressed(size(cm_A), cm_A)
        As = A*scalar
        if are_equal(A*scalar, built_mult)
            correct_outputs += 1
        else
            incorrect_outputs += 1    
        end
    end

    return correct_outputs, incorrect_outputs
    
end


function run_compressed_scalar_multiplication_tests()
    low_m_for_test, high_m_for_test, low_n_for_test, high_n_for_test = 15, 50, 15, 50
    a_for_test, b_for_test = 0, 50
    correct, incorrect = test_compressed_matrix_scalar_multiplication(low_m_for_test, high_m_for_test, low_n_for_test, high_n_for_test, a_for_test, b_for_test, 15, 1, 0.1)
    println(correct, " ", incorrect)
end


function test_compressed_matrix_subtraction(low_m, high_m, low_n, high_n, a, b, tests, gamma, eps)
    correct_outputs = 0
    incorrect_outputs = 0
    for _ in 1:1:tests 
        m, n = rand(low_m:1:high_m), rand(low_n:1:high_n)
        A = generate_random_matrix(m, n, a, b)
        B = generate_random_matrix(m, n, a, b)
        cm_A = perform_matrix_compression(A, gamma=gamma, eps=eps)
        cm_B = perform_matrix_compression(B, gamma=gamma, eps=eps)
        cm_sub = cm_A - cm_B
        AB = A-B
        built_sub = build_matrix_from_compressed(size(cm_sub), cm_sub)
        if are_equal(A-B, built_sub)
            correct_outputs += 1
        else 
            incorrect_outputs += 1    
        end
    end

    return correct_outputs, incorrect_outputs
end


function run_compressed_subtraction_tests()
    low_m_for_test, high_m_for_test, low_n_for_test, high_n_for_test = 15, 50, 15, 50
    a_for_test, b_for_test = 0, 50
    correct, incorrect = test_compressed_matrix_subtraction(low_m_for_test, high_m_for_test, low_n_for_test, high_n_for_test, a_for_test, b_for_test, 15, 1, 0.1)
    println(correct, " ", incorrect)
end


function test_compressed_matrix_vector_multiplication(low_m, high_m, low_n, high_n, a, b, tests, gamma, eps)
    correct_outputs = 0
    incorrect_outputs = 0

    for _ in 1:1:tests 
        m, n = rand(low_m:1:high_m), rand(low_n:1:high_n)
        A = generate_random_matrix(m, n, a, b)
        X = generate_random_vector(n, a, b)
        v = perform_matrix_compression(A, gamma=gamma, eps=eps)
        Y = v * X
        if are_equal(A*X, Y)
            correct_outputs += 1
        else
            incorrect_outputs += 1    
        end
    end

    return correct_outputs, incorrect_outputs
end


function run_compressed_matrix_vector_multiplication_tests()
    low_m_for_test, high_m_for_test, low_n_for_test, high_n_for_test = 15, 50, 15, 50
    a_for_test, b_for_test = 1, 50
    correct, incorrect = test_compressed_matrix_vector_multiplication(low_m_for_test, high_m_for_test, low_n_for_test, high_n_for_test, a_for_test, b_for_test, 15, 1, 0.1)
    println(correct, " ", incorrect)
end

In [ ]:
run_compression_tests()
run_compressed_addition_tests()
run_compressed_multiplication_tests()
run_compressed_scalar_multiplication_tests()
run_compressed_subtraction_tests()
run_compressed_matrix_vector_multiplication_tests()

In [11]:
function compressed_matrix_drawer(target_matrix, node)
    r_min, r_max, c_min, c_max = node.addr
    
    target_matrix[r_min:r_max, c_min:c_max] .= 1
    if !isnothing(node.rank) && node.rank > 0
        for i in r_min:r_max
            target_matrix[i:i, c_min:c_min+node.rank - 1] .= 0
        end

        for j in c_min:c_max
            target_matrix[r_min:r_min+node.rank - 1, j:j] .= 0
        end
    end

    if !isnothing(node.left_upper_child)
        compressed_matrix_drawer(target_matrix, node.left_upper_child)
    end

    if !isnothing(node.right_upper_child)
        compressed_matrix_drawer(target_matrix, node.right_upper_child)
    end
    
    if !isnothing(node.left_lower_child)
        compressed_matrix_drawer(target_matrix, node.left_lower_child)
    end
    
    if !isnothing(node.right_lower_child)
        compressed_matrix_drawer(target_matrix, node.right_lower_child)
    end
    
end;

In [ ]:
filenames = ["cm_2^6.png", "cm_2^9.png", "cm_2^12.png"]
ns = [2, 3, 4]
times_MM = []
times_MV = []


for k in 2:4
    println("k = ", k)
    M = generate_random_matrix(2^(3*k), 2^(3*k), 0, 1)
    V = generate_random_vector(2^(3*k), 0, 1)

    CM = perform_matrix_compression(M, gamma=1, eps=0.1)

    # save compressed matrix drawing
    drawn_matrix = zeros(size(CM))
    compressed_matrix_drawer(drawn_matrix, CM.head)
    save(filenames[k-1], drawn_matrix)

    tik = time()
    CMV = CM * V
    tok = time()

    push!(times_MV, tok-tik)

    # println(M * V)
    # println(CMV)

    tik2 = time()
    CM2 = CM * CM
    tok2 = time()

    push!(times_MM, tok2-tik2)


    # println(M * M)
    # println(build_matrix_from_compressed(size(CM2), CM2))

end

In [ ]:
load(filenames[1])

In [ ]:
load(filenames[2])

In [ ]:
load(filenames[3])

In [ ]:
plot(ns, times_MV, label="Matrix vector multiplication", xlabel="2^(3*k)", ylabel="Time (s)", title="Time vs k (Matrix vector multiplication)")

In [ ]:
plot(ns, times_MM, label="Matrix matrix multiplication", xlabel="2^(3*k)", ylabel="Time (s)", title="Time vs k (Matrix matrix multiplication)")